###### *Adaptação de artigo publicado em* https://stackabuse.com/creating-a-simple-recommender-system-in-python-using-pandas/

##### Vamos construir um simples **Sistema de Recomendação** de Filmes, utilizando a correlação entre as classificações que os usuários deram para cada Filme.

In [1]:
import numpy as np
import pandas as pd

In [2]:
urlDataSet = 'https://raw.githubusercontent.com/EngenhariaDeComputacaoReinaldo/miniCursoPandas/master/ml-latest-small/'

In [3]:
ratings_data = pd.read_csv(urlDataSet+'ratings.csv')
movie_names = pd.read_csv(urlDataSet+'movies.csv')

In [62]:
ratings_data.head(10)

In [63]:
movie_names.head(10)

#### Gerando um novo DataSet com os dados agrupados de Ratings e Movies

In [11]:
movie_data = pd.merge(ratings_data, movie_names, on='movieId')

In [64]:
movie_data.head(10)

In [65]:
print(len(ratings_data),len(movie_names), len(movie_data))

Agora vamos dar uma olhada na classificação média de cada filme. Para fazer isso, podemos agrupar o conjunto de dados pelo título do filme e, em seguida, calcular a média da classificação de cada filme. Por fim, exibiremos os cinco primeiros filmes junto com sua classificação média.

In [66]:
movie_data.groupby('title')['rating'].mean().head(10)

Você pode ver que as avaliações médias não são classificadas. Vamos classificar as avaliações na ordem decrescente de suas avaliações médias:

In [67]:
movie_data.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)

Os filmes agora foram classificados de acordo com a ordem crescente de suas classificações. No entanto, existe um problema. Um filme pode chegar ao topo da lista acima, mesmo que apenas um único usuário tenha dado cinco estrelas. Portanto, as estatísticas acima podem ser enganosas. Normalmente, um filme que é realmente bom recebe uma classificação mais alta por um grande número de usuários.

In [68]:
movie_data.groupby('title')['rating'].count().sort_values(ascending=False).head(10)

Agora você pode ver alguns filmes realmente bons no topo. A lista acima confirma nossa afirmação de que bons filmes normalmente recebem classificações mais altas. Agora sabemos que a avaliação média por filme e o número de classificações por filme são atributos importantes. Vamos criar um novo dataframe que contenha esses dois atributos.

Criando um DataFrame com Mean and Count dos Ratings dos Filmes

In [69]:
ratings_mean_count = pd.DataFrame(movie_data.groupby('title')['rating'].mean())
ratings_mean_count.head(5)

In [70]:
ratings_mean_count['rating_counts'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())
ratings_mean_count.head(5)

In [19]:
ratings_mean_count = ratings_mean_count.rename(index=str, columns={"rating": "rating_means"})

In [71]:
ratings_mean_count.head(5)

Você pode ver o título do filme, juntamente com a ratings média e o número de ratings do filme.

Vamos traçar um histograma para o número de avaliações representadas pela coluna "rating_counts" no dataframe acima. Execute o seguinte script:

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('dark')
%matplotlib inline

In [72]:
plt.figure(figsize=(10,8))

plt.ylabel('Qtd de Filmes')
plt.xlabel('Qtd de ratings')

ratings_mean_count['rating_counts'].hist(bins=50)

A partir da saída, você pode ver que a maioria dos filmes recebeu menos de 50 classificações. Enquanto o número de filmes com mais de 100 classificações é muito baixo.

Agora vamos traçar um histograma para as classificações médias:

In [73]:
plt.figure(figsize=(10,8))

plt.ylabel('Qtd de Filmes')
plt.xlabel('Média de ratings')

ratings_mean_count['rating_means'].hist(bins=50)

Você pode ver que os valores inteiros têm barras mais altas do que os valores flutuantes, já que a maioria dos usuários atribui a classificação como valor inteiro, isto é, 1, 2, 3, 4 ou 5.

Anteriormente, dissemos que filmes com um número maior de avaliações geralmente têm uma classificação média alta, já que um filme bom é normalmente conhecido e um filme conhecido é assistido por um grande número de pessoas e, portanto, geralmente tem Avaliação. Vamos ver se esse também é o caso dos filmes em nosso conjunto de dados. Vamos traçar as avaliações médias em relação ao número de classificações:

In [74]:
plt.figure(figsize=(12,6))
sns.jointplot(x='rating_means', y='rating_counts', data=ratings_mean_count, alpha=0.4)

O gráfico mostra que, em geral, os filmes com classificações médias mais altas têm, na verdade, maior número de classificações, em comparação com os filmes com classificações médias mais baixas.


## **Similaridade Entre Filmes**

Vamos usar a correlação entre as avaliações de um filme como a métrica de similaridade. Para encontrar a correlação entre as classificações do filme, precisamos criar uma matriz em que cada coluna seja um nome de filme e cada linha contenha a classificação atribuída por um usuário específico a esse filme. Tenha em mente que essa matriz terá muitos valores nulos, pois cada filme não é classificado por todos os usuários.

In [75]:
movie_data.head(5)

In [53]:
user_movie_rating = movie_data.pivot_table(index='userId', columns='title', values='rating')

In [76]:
user_movie_rating.head(10)

Sabemos que cada coluna contém todas as classificações de usuários de um filme específico. Vamos encontrar todas as avaliações do usuário para o filme "Forrest Gump (1994)" e encontrar os filmes semelhantes a ele. Escolhemos este filme porque tem o maior número de classificações e queremos encontrar a correlação entre os filmes que têm um maior número de classificações.

In [55]:
forrest_gump_ratings = user_movie_rating['Forrest Gump (1994)']

In [77]:
forrest_gump_ratings.head(20)

Agora vamos recuperar todos os filmes que são semelhantes a "Forrest Gump (1994)". Podemos encontrar a correlação (Pearson) entre as classificações de usuários para "Forest Gump (1994)" e todos os outros filmes usando a função corrwith()

**Coeficiente de correlação de Pearson**

*O Coeficiente de correlação de Pearson (r) é uma medida adimensional que pode assumir valores no intervalo entre -1 e +1.*

*O coeficiente mede a intensidade e a direção de relações lineares. A intensidade diz respeito ao grau de relacionamento entre duas variáveis. Quanto mais próximo dos extremos do intervalo, (-1 e +1) mais forte é a correlação.*

*Quanto mais próximo do centro do intervalo, zero, mas fraca é a correlação linear.*

*A direção diz respeito ao tipo de correlação. Correlação positiva ou direta `(r>0) `representa que os valores altos de uma variável correspondem a valores altos da outra variável.*

*Correlação negativa ou inversa representa que valores altos de uma das variáveis correspondem a valores baixos de outra.*

*Pode-se supor a título de exemplo, que entre as variáveis peso da carga de um caminhão e consumo de combustível o coeficiente r=+0,80. Indicando uma correlação direta, quanto maior o peso maior o consumo e moderada forte uma vez que 0,80 está próximo do extremo +1.*

*Em outro exemplo, pode-se supor que a quantidade de borrachas em pneus e a quilometragem por eles percorrida resulte num coeficiente r=-0,92. Indicando que há uma correlação inversa, quanto maior a quilometragem maior o desgaste e forte (valor de -0,92 está muito próximo de -1).*

fonte: *http://www5.eesc.usp.br/saate/index.php/saate/Indicar-a-T%C3%A9cnica/Associar/2.-%C3%81rvore-de-decis%C3%A3o/Gloss%C3%A1rio/Coeficiente-de-correla%C3%A7%C3%A3o-de-Pearson#:~:text=A%20intensidade%20diz%20respeito%20ao,fraca%20%C3%A9%20a%20correla%C3%A7%C3%A3o%20linear.*

In [78]:
movies_like_forest_gump = user_movie_rating.corrwith(forrest_gump_ratings)

In [79]:
corr_forrest_gump = pd.DataFrame(movies_like_forest_gump, columns=['Correlation'])
corr_forrest_gump.dropna(inplace=True)
corr_forrest_gump.head()

No script acima, primeiro recuperamos a lista de todos os filmes relacionados a "Forrest Gump (1994)", juntamente com seu valor de correlação, usando a função corrwith(). Em seguida, criamos um dataframe que contém as colunas de título e correlação do filme. Em seguida, removemos todos os valores de NA do dataframe e exibimos suas primeiras 5 linhas usando a função head().

Vamos classificar os filmes em ordem descendente de correlação para ver filmes altamente correlacionados no topo

In [80]:
corr_forrest_gump.sort_values('Correlation', ascending=False).head(10)

A partir da saída, você pode ver que os filmes que têm alta correlação com "Forrest Gump (1994)" não são muito conhecidos. Isso mostra que a correlação sozinha não é uma boa métrica para a similaridade, porque pode haver um usuário que assistiu "Forest Gump (1994)" e apenas um outro filme e classificou os dois como 5.

Uma solução para esse problema é recuperar apenas os filmes correlacionados que tenham pelo menos mais de 50 classificações. Para fazer isso, adicione a coluna rating_counts do dataframe rating_mean_count ao nosso corr_forrest_gump dataframe. Execute o seguinte script para fazer isso:

In [81]:
corr_forrest_gump = corr_forrest_gump.join(ratings_mean_count['rating_counts'])
corr_forrest_gump.head()

Você pode ver que o filme "...And Justice for All (1979)", que tem a maior correlação, tem apenas três avaliações. Isso significa que apenas três usuários deram as mesmas classificações para **"Forest Gump (1994)"** e **"...And Justice for All (1979)"**, no entanto, podemos deduzir que um filme não pode ser declarado semelhante ao outro filme baseado em apenas 3 classificações. É por isso que adicionamos a coluna "rating_counts". Vamos agora filtrar filmes correlacionados a "Forest Gump (1994)", que têm mais de 50 avaliações.

In [82]:
corr_forrest_gump[corr_forrest_gump ['rating_counts']>50].sort_values('Correlation', ascending=False).head(10)

Agora você pode ver na saída os filmes que estão altamente correlacionados com "Forrest Gump (1994)". Os filmes da lista são alguns dos mais famosos filmes de Hollywood, e desde que "Forest Gump (1994)" também é um filme muito famoso, há uma grande chance de que esses filmes sejam correlacionados.

## Que filmes você recomendaria para usuários que gostaram de assistir os seguintes filmes:

### 1 - Silence of the Lambs, The (1991)

### 2 - Pulp Fiction (1994)

### 3 - Matrix, The (1999)

### 4 - Star Wars: Episode IV - A New Hope (1977)

### 5 - Schindler's List (1993)